In [1]:
# Situação da carteira de Ozelina Gusmão - 2026-01-19
INV_Total = 169626.87
QBTC11 = 3268.08

Saldo_OBJETIVO = INV_Total - QBTC11
Saldo_OBJETIVO

166358.79

### 1 - Imports

In [2]:
# =========================
# Célula 1 — Imports
# =========================
import numpy as np  # cálculos numéricos
import pandas as pd  # tabelas para inspeção/relatório
import plotly.graph_objects as go  # gráficos interativos (hover)

from datetime import datetime  # manipulação de datas
from datetime import timedelta # manipulação de datas


### 2 — Entradas do usuário (EDITE AQUI)

In [3]:
# =========================
# Célula 2 — Entradas do usuário (EDITE AQUI)
# =========================

# --- Taxa de juros ---
taxa_tipo = "anual"  # "mensal" ou "anual"
i_input = 0.14        # se mensal: 0.01 = 1% a.m.; se anual: 0.12 = 12% a.a.

# --- Recursos disponíveis ---
C1 = 166358.79          # capital inicial no investimento 1 (tempo 0)
C2 = 0.0              # capital inicial no investimento 2 (tempo 0) — na sua folha é 0

AxT = 57111.68+ 18148.33         # aporte extra total (lump sum no tempo 0) para dividir entre inv1 e inv2
aT = 7500.0           # aporte mensal total para dividir entre inv1 e inv2

# --- Metas ---
M1_meta = 450000.0     # meta do investimento 1
M2_meta = 45000.0     # meta do investimento 2

# --- Parâmetros do solver ---
max_meses = 6000      # limite de simulação (meses)
Na = 61               # resolução do grid para a1 (coarse)
Nx = 61               # resolução do grid para Ax1 (coarse)
refino_passos = 3     # quantas rodadas de refino local (grid menor ao redor do ótimo)
refino_fator = 0.25   # tamanho da "janela" de refino (fração do intervalo)

# --- Preferências de exibição ---
mostrar_heatmap = True    # True para exibir mapa (a1 x Ax1) do tempo total
mostrar_frontier = True   # True para exibir curva "best time vs a1" (com Ax1 ótimo por a1)

# --- Data do usuário para cálculo da data prevista ---
data_usuario_str = "2026-01-19"  # exemplo de data fornecida pelo usuário
data_usuario = datetime.strptime(data_usuario_str, "%Y-%m-%d")


### 3 — Conversão de taxa (mensal) e validações

In [4]:
# =========================
# Célula 3 — Conversão de taxa (mensal) e validações
# =========================

# Converte taxa anual para mensal equivalente (juros compostos)
if taxa_tipo.lower() == "anual":  # se o usuário informou taxa anual
    i = (1.0 + i_input) ** (1.0 / 12.0) - 1.0  # taxa mensal equivalente
else:  # se já for mensal
    i = float(i_input)  # garante tipo float

# Validações básicas
assert i >= 0.0, "A taxa i deve ser >= 0."
assert aT >= 0.0, "Aporte mensal total aT deve ser >= 0."
assert AxT >= 0.0, "Aporte extra total AxT deve ser >= 0."
assert C1 >= 0.0 and C2 >= 0.0, "Capitais iniciais devem ser >= 0."
assert M1_meta >= 0.0 and M2_meta >= 0.0, "Metas devem ser >= 0."


### 4 — Simulação com transferência de aporte (core)

In [5]:
# =========================
# Célula 4 — Simulação com transferência de aporte (core)
# =========================

def simular_duas_metas(
    i,               # taxa mensal
    C1, C2,          # capitais iniciais (tempo 0)
    Ax1, Ax2,        # aportes extras (tempo 0) em cada investimento
    a1, a2,          # aportes mensais iniciais (somam aT)
    M1, M2,          # metas
    max_meses=6000   # limite de simulação
):
    """
    Simula mês a mês:
      - saldo_t+1 = saldo_t*(1+i) + aporte_mensal
      - quando uma meta é atingida, o aporte mensal daquele investimento é transferido integralmente ao outro
        a partir do mês seguinte (implementado ao atualizar a taxa a1/a2 após a detecção).
    Retorna:
      - t1, t2 (meses em que bateu cada meta; 0 se já bate no t=0; np.inf se não bateu)
      - dataframe com trajetória para plot (saldos e aportes por mês)
    """

    # Saldo inicial em t=0 (inclui capital inicial + aporte extra)
    b1 = float(C1 + Ax1)  # saldo investimento 1
    b2 = float(C2 + Ax2)  # saldo investimento 2

    # Aportes mensais "correntes" (podem mudar após transferência)
    a1_cur = float(a1)  # aporte atual no inv1
    a2_cur = float(a2)  # aporte atual no inv2

    # Flags de conclusão e tempos
    done1 = (b1 >= M1)  # se já atingiu meta 1 no tempo 0
    done2 = (b2 >= M2)  # se já atingiu meta 2 no tempo 0
    t1 = 0 if done1 else np.inf  # tempo de conclusão da meta 1
    t2 = 0 if done2 else np.inf  # tempo de conclusão da meta 2

    # Se já concluiu um investimento em t=0, transfere o aporte mensal dele imediatamente
    if done1 and a1_cur > 0.0:           # se meta1 já foi batida e havia aporte em inv1
        a2_cur += a1_cur                 # transfere para inv2
        a1_cur = 0.0                     # zera inv1

    if done2 and a2_cur > 0.0:           # se meta2 já foi batida e havia aporte em inv2
        a1_cur += a2_cur                 # transfere para inv1
        a2_cur = 0.0                     # zera inv2

    # Armazenamento da trajetória (para plot/hover)
    meses = [0]                           # eixo x: meses
    saldo1 = [b1]                         # saldo inv1
    saldo2 = [b2]                         # saldo inv2
    aporte1 = [a1_cur]                    # aporte aplicado (próximo ciclo) — informativo
    aporte2 = [a2_cur]                    # aporte aplicado (próximo ciclo) — informativo
    done1_list = [done1]                  # status meta1
    done2_list = [done2]                  # status meta2

    # Se ambos já concluíram em t=0, retorna cedo
    if done1 and done2:  # ambos concluídos
        df = pd.DataFrame({
            "mes": meses, "saldo1": saldo1, "saldo2": saldo2,
            "aporte1": aporte1, "aporte2": aporte2,
            "done1": done1_list, "done2": done2_list
        })
        return t1, t2, df

    # Simulação mês a mês
    for m in range(1, max_meses + 1):     # meses 1..max_meses
        # Aplica juros e aporte mensal (aporte no fim do mês)
        b1 = b1 * (1.0 + i) + a1_cur      # evolução inv1
        b2 = b2 * (1.0 + i) + a2_cur      # evolução inv2

        # Verifica se atingiu meta 1 neste mês (se ainda não tinha atingido)
        if (not done1) and (b1 >= M1):    # se cruzou a meta
            done1 = True                  # marca concluído
            t1 = m                        # registra tempo
            # transfere o aporte mensal do inv1 para inv2 (a partir do próximo mês)
            if a1_cur > 0.0:              # se havia aporte
                a2_cur += a1_cur          # soma no outro
                a1_cur = 0.0              # zera este

        # Verifica se atingiu meta 2 neste mês (se ainda não tinha atingido)
        if (not done2) and (b2 >= M2):    # se cruzou a meta
            done2 = True                  # marca concluído
            t2 = m                        # registra tempo
            # transfere o aporte mensal do inv2 para inv1 (a partir do próximo mês)
            if a2_cur > 0.0:              # se havia aporte
                a1_cur += a2_cur          # soma no outro
                a2_cur = 0.0              # zera este

        # Salva estado do mês (para plot)
        meses.append(m)                    # mês atual
        saldo1.append(b1)                  # saldo inv1
        saldo2.append(b2)                  # saldo inv2
        aporte1.append(a1_cur)             # aporte que vigorará no próximo mês
        aporte2.append(a2_cur)             # aporte que vigorará no próximo mês
        done1_list.append(done1)           # status
        done2_list.append(done2)           # status

        # Se ambos concluíram, interrompe
        if done1 and done2:                # se ambas metas batidas
            break                          # encerra simulação

    # Constrói DataFrame final
    df = pd.DataFrame({
        "mes": meses, "saldo1": saldo1, "saldo2": saldo2,
        "aporte1": aporte1, "aporte2": aporte2,
        "done1": done1_list, "done2": done2_list
    })

    # Se alguma meta não foi atingida, mantém np.inf
    if not done1:  # meta 1 não atingiu
        t1 = np.inf
    if not done2:  # meta 2 não atingiu
        t2 = np.inf

    return t1, t2, df


### 5 — Função objetivo (tempo total) e avaliação em grid

In [6]:
# =========================
# Célula 5 — Função objetivo (tempo total) e avaliação em grid
# =========================

def avaliar_tempo_total(i, C1, C2, AxT, aT, M1, M2, a1, Ax1, max_meses):
    """
    Dado a1 e Ax1, define:
      a2 = aT - a1
      Ax2 = AxT - Ax1
    e calcula t_total = max(t1,t2) pela simulação com transferência.
    Retorna (t_total, t1, t2).
    """
    # Impõe as variáveis derivadas
    a2 = aT - a1                  # aporte do investimento 2
    Ax2 = AxT - Ax1               # extra do investimento 2

    # Checa limites (viabilidade)
    if a1 < 0.0 or a2 < 0.0:      # aportes não-negativos
        return np.inf, np.inf, np.inf
    if Ax1 < 0.0 or Ax2 < 0.0:    # extras não-negativos
        return np.inf, np.inf, np.inf

    # Simula a dinâmica com transferência
    t1, t2, _ = simular_duas_metas(
        i=i, C1=C1, C2=C2,
        Ax1=Ax1, Ax2=Ax2,
        a1=a1, a2=a2,
        M1=M1, M2=M2,
        max_meses=max_meses
    )

    # Tempo total é o makespan
    t_total = max(t1, t2)         # objetivo
    return t_total, t1, t2


def grid_search(i, C1, C2, AxT, aT, M1, M2, max_meses, Na=61, Nx=61, a1_bounds=None, Ax1_bounds=None):
    """
    Busca em grid (coarse) no retângulo [a1_min,a1_max] x [Ax1_min,Ax1_max].
    Retorna: melhor ponto e tabelas com resultados.
    """

    # Define bounds padrão
    a1_min, a1_max = (0.0, aT) if a1_bounds is None else a1_bounds      # limites de a1
    Ax1_min, Ax1_max = (0.0, AxT) if Ax1_bounds is None else Ax1_bounds  # limites de Ax1

    # Gera grids
    a1_grid = np.linspace(a1_min, a1_max, Na)    # valores de a1
    Ax1_grid = np.linspace(Ax1_min, Ax1_max, Nx) # valores de Ax1

    # Matriz para guardar t_total
    T = np.full((Na, Nx), np.inf, dtype=float)   # matriz de tempos totais
    T1 = np.full((Na, Nx), np.inf, dtype=float)  # matriz de t1
    T2 = np.full((Na, Nx), np.inf, dtype=float)  # matriz de t2

    # Melhor registro
    best = {"t_total": np.inf, "a1": None, "Ax1": None, "t1": None, "t2": None}

    # Varre o grid
    for ia, a1 in enumerate(a1_grid):            # loop em a1
        for ix, Ax1 in enumerate(Ax1_grid):      # loop em Ax1
            t_total, t1, t2 = avaliar_tempo_total(
                i=i, C1=C1, C2=C2, AxT=AxT, aT=aT,
                M1=M1, M2=M2,
                a1=float(a1), Ax1=float(Ax1),
                max_meses=max_meses
            )
            T[ia, ix] = t_total                  # salva tempo total
            T1[ia, ix] = t1                      # salva t1
            T2[ia, ix] = t2                      # salva t2

            # Atualiza melhor
            if t_total < best["t_total"]:         # se achou tempo menor
                best = {"t_total": t_total, "a1": float(a1), "Ax1": float(Ax1), "t1": t1, "t2": t2}

    return best, a1_grid, Ax1_grid, T, T1, T2


### 6 — Otimização: grid coarse + refino local

In [7]:
# =========================
# Célula 6 — Otimização: grid coarse + refino local
# =========================

# 1) Busca coarse no domínio todo
best, a1_grid, Ax1_grid, T, T1, T2 = grid_search(
    i=i, C1=C1, C2=C2, AxT=AxT, aT=aT,
    M1=M1_meta, M2=M2_meta,
    max_meses=max_meses,
    Na=Na, Nx=Nx
)

# 2) Refino local iterativo ao redor do melhor ponto encontrado
for k in range(refino_passos):
    # Janela de refino (encolhe a cada passo)
    janela_a = (aT * (refino_fator ** (k + 1)))  # largura em a1
    janela_Ax = (AxT * (refino_fator ** (k + 1))) # largura em Ax1

    # Bounds locais ao redor do best atual
    a1_min = max(0.0, best["a1"] - janela_a)     # mínimo local de a1
    a1_max = min(aT,  best["a1"] + janela_a)     # máximo local de a1
    Ax1_min = max(0.0, best["Ax1"] - janela_Ax)  # mínimo local de Ax1
    Ax1_max = min(AxT, best["Ax1"] + janela_Ax)  # máximo local de Ax1

    # Executa grid local mais denso (pode ajustar densidade)
    best_local, a1g, Ax1g, Tg, T1g, T2g = grid_search(
        i=i, C1=C1, C2=C2, AxT=AxT, aT=aT,
        M1=M1_meta, M2=M2_meta,
        max_meses=max_meses,
        Na=max(41, Na//2), Nx=max(41, Nx//2),
        a1_bounds=(a1_min, a1_max),
        Ax1_bounds=(Ax1_min, Ax1_max)
    )

    # Se melhorou, atualiza
    if best_local["t_total"] < best["t_total"]:
        best = best_local  # substitui melhor global

# Deriva variáveis complementares
a1_opt = best["a1"]                 # a1 ótimo
a2_opt = aT - a1_opt                # a2 ótimo
Ax1_opt = best["Ax1"]               # Ax1 ótimo
Ax2_opt = AxT - Ax1_opt             # Ax2 ótimo
t1_opt = best["t1"]                 # tempo de meta1 (meses)
t2_opt = best["t2"]                 # tempo de meta2 (meses)
t_total_opt = best["t_total"]       # tempo total ótimo (meses)

best


{'t_total': 23, 'a1': 125.0, 'Ax1': 67734.009, 't1': 23, 't2': 5}

### 7 — Trajetória no ótimo (para plot principal)

In [8]:
# =========================
# Célula 7 — Trajetória no ótimo (para plot principal)
# =========================

t1_sim, t2_sim, df_traj = simular_duas_metas(
    i=i, C1=C1, C2=C2,
    Ax1=Ax1_opt, Ax2=Ax2_opt,
    a1=a1_opt, a2=a2_opt,
    M1=M1_meta, M2=M2_meta,
    max_meses=max_meses
)

# Confirma consistência
t_total_sim = max(t1_sim, t2_sim)  # tempo total via simulação
t_total_sim, t1_sim, t2_sim


(23, 23, 5)

### 8 — Plot 1: Evolução dos saldos (hover rico)

In [9]:
# =========================
# Célula 8 — Plot 1: Evolução dos saldos (hover rico)
# =========================

# Prepara dados do dataframe
x = df_traj["mes"].values                         # meses
y1 = df_traj["saldo1"].values                     # saldo inv1
y2 = df_traj["saldo2"].values                     # saldo inv2
ap1 = df_traj["aporte1"].values                   # aporte corrente inv1
ap2 = df_traj["aporte2"].values                   # aporte corrente inv2
tot = y1 + y2                                     # saldo total

# Customdata para hover (cada ponto carrega várias colunas)
custom1 = np.stack([ap1, y1, y2, tot], axis=1)    # dados para hover da série 1
custom2 = np.stack([ap2, y1, y2, tot], axis=1)    # dados para hover da série 2

fig = go.Figure()  # cria figura

# Linha do investimento 1
fig.add_trace(go.Scatter(
    x=x, y=y1, mode="lines",
    name="Investimento 1 (saldo)",
    customdata=custom1,
    hovertemplate=(
        "Mês: %{x}<br>"
        "Saldo Inv1: %{y:,.2f}<br>"
        "Aporte Inv1 (próx mês): %{customdata[0]:,.2f}<br>"
        "Saldo Inv2: %{customdata[2]:,.2f}<br>"
        "Total: %{customdata[3]:,.2f}<extra></extra>"
    )
))

# Linha do investimento 2
fig.add_trace(go.Scatter(
    x=x, y=y2, mode="lines",
    name="Investimento 2 (saldo)",
    customdata=custom2,
    hovertemplate=(
        "Mês: %{x}<br>"
        "Saldo Inv2: %{y:,.2f}<br>"
        "Aporte Inv2 (próx mês): %{customdata[0]:,.2f}<br>"
        "Saldo Inv1: %{customdata[1]:,.2f}<br>"
        "Total: %{customdata[3]:,.2f}<extra></extra>"
    )
))

# Linha do total (opcional)
fig.add_trace(go.Scatter(
    x=x, y=tot, mode="lines",
    name="Total (Inv1 + Inv2)",
    hovertemplate="Mês: %{x}<br>Total: %{y:,.2f}<extra></extra>"
))

# Linhas horizontais das metas
fig.add_hline(y=M1_meta, line_dash="dash", annotation_text="Meta M1", annotation_position="top left")
fig.add_hline(y=M2_meta, line_dash="dash", annotation_text="Meta M2", annotation_position="bottom left")

# Marcadores nos meses de conclusão
if np.isfinite(t1_sim):
    fig.add_vline(x=int(t1_sim), line_dash="dot", annotation_text=f"t1={int(t1_sim)}m", annotation_position="top")
if np.isfinite(t2_sim):
    fig.add_vline(x=int(t2_sim), line_dash="dot", annotation_text=f"t2={int(t2_sim)}m", annotation_position="bottom")

# Layout
fig.update_layout(
    title="Evolução dos saldos com transferência de aporte (solução ótima)",
    xaxis_title="Tempo (meses)",
    yaxis_title="Saldo",
    hovermode="x unified"
)

fig.show()


### 9 — Plot 2 (opcional): Heatmap do tempo total em função de (a1, Ax1)

In [10]:
# =========================
# Célula 9 — Plot 2 (opcional): Heatmap do tempo total em função de (a1, Ax1)
# =========================

if mostrar_heatmap:
    # Para heatmap, usamos o grid coarse inicial (a1_grid, Ax1_grid, T)
    # Observação: heatmap espera z com shape (len(y), len(x)) se usarmos eixos invertidos;
    # aqui manteremos: linhas=a1, colunas=Ax1.
    A1, AX1 = np.meshgrid(Ax1_grid, a1_grid)  # malha para customdata

    # Customdata no heatmap para mostrar a1, a2, Ax1, Ax2 ao passar mouse
    a2_mat = aT - AX1*0 + (aT - a1_grid[:, None])  # constrói matriz a2 (broadcast)
    Ax2_mat = AxT - A1                             # matriz Ax2

    # Empilha customdata por célula (a1, a2, Ax1, Ax2)
    custom_hm = np.stack([AX1, a2_mat, A1, Ax2_mat], axis=2)

    fig_hm = go.Figure(data=go.Heatmap(
        z=T, x=Ax1_grid, y=a1_grid,
        customdata=custom_hm,
        hovertemplate=(
            "a1: %{customdata[0]:,.2f}<br>"
            "a2: %{customdata[1]:,.2f}<br>"
            "Ax1: %{customdata[2]:,.2f}<br>"
            "Ax2: %{customdata[3]:,.2f}<br>"
            "Tempo total (meses): %{z:.0f}<extra></extra>"
        )
    ))

    # Marca o ótimo no heatmap
    fig_hm.add_trace(go.Scatter(
        x=[Ax1_opt], y=[a1_opt],
        mode="markers",
        name="Ótimo",
        marker=dict(size=10, symbol="x"),
        hovertemplate=(
            "ÓTIMO<br>"
            f"a1={a1_opt:,.2f}<br>a2={a2_opt:,.2f}<br>"
            f"Ax1={Ax1_opt:,.2f}<br>Ax2={Ax2_opt:,.2f}<br>"
            f"t_total={t_total_opt:.0f} meses<extra></extra>"
        )
    ))

    fig_hm.update_layout(
        title="Mapa do tempo total (meses) em função de (a1, Ax1) — grid coarse",
        xaxis_title="Ax1 (aporte extra no inv1)",
        yaxis_title="a1 (aporte mensal no inv1)"
    )

    fig_hm.show()


### 10 — Plot 3 (opcional): Curva (fronteira) "melhor tempo vs a1" com Ax1 ótimo por a1

In [11]:
# =========================
# Célula 10 — Plot 3 (opcional): Curva (fronteira) "melhor tempo vs a1" com Ax1 ótimo por a1
# =========================

if mostrar_frontier:
    # Grid de a1 para construir a curva
    a1_line = np.linspace(0.0, aT, 101)                 # varre a1
    best_times = []                                     # melhor tempo para cada a1
    best_Ax1_for_a1 = []                                # Ax1 que minimiza para cada a1
    best_t1_for_a1 = []                                 # t1 correspondente
    best_t2_for_a1 = []                                 # t2 correspondente

    # Grid de Ax1 para cada a1 (pode ajustar resolução)
    Ax1_line_grid = np.linspace(0.0, AxT, 121)          # varre Ax1

    for a1_val in a1_line:                               # para cada a1
        tbest = np.inf                                   # melhor tempo inicial
        Axbest = 0.0                                     # melhor Ax1 inicial
        t1best = np.inf                                  # melhor t1
        t2best = np.inf                                  # melhor t2

        for Ax1_val in Ax1_line_grid:                    # varre Ax1
            t_total, t1v, t2v = avaliar_tempo_total(
                i=i, C1=C1, C2=C2, AxT=AxT, aT=aT,
                M1=M1_meta, M2=M2_meta,
                a1=float(a1_val), Ax1=float(Ax1_val),
                max_meses=max_meses
            )
            if t_total < tbest:                           # se melhora
                tbest = t_total                           # atualiza melhor
                Axbest = float(Ax1_val)                   # guarda Ax1
                t1best = t1v                              # guarda t1
                t2best = t2v                              # guarda t2

        best_times.append(tbest)                           # registra melhor tempo para esse a1
        best_Ax1_for_a1.append(Axbest)                     # registra Ax1 ótimo
        best_t1_for_a1.append(t1best)                      # registra t1
        best_t2_for_a1.append(t2best)                      # registra t2

    # Converte para arrays
    best_times = np.array(best_times)                      # tempos
    best_Ax1_for_a1 = np.array(best_Ax1_for_a1)            # Ax1*
    best_t1_for_a1 = np.array(best_t1_for_a1)              # t1*
    best_t2_for_a1 = np.array(best_t2_for_a1)              # t2*

    # Customdata para hover na curva: a2, Ax1*, Ax2*, t1*, t2*
    a2_line = aT - a1_line                                 # a2 derivado
    Ax2_line = AxT - best_Ax1_for_a1                        # Ax2 derivado
    custom_curve = np.stack([a2_line, best_Ax1_for_a1, Ax2_line, best_t1_for_a1, best_t2_for_a1], axis=1)

    fig_curve = go.Figure()

    fig_curve.add_trace(go.Scatter(
        x=a1_line, y=best_times, mode="lines+markers",
        name="Melhor tempo total dado a1 (otimizando Ax1)",
        customdata=custom_curve,
        hovertemplate=(
            "a1: %{x:,.2f}<br>"
            "Tempo total (meses): %{y:.0f}<br>"
            "a2: %{customdata[0]:,.2f}<br>"
            "Ax1*: %{customdata[1]:,.2f}<br>"
            "Ax2*: %{customdata[2]:,.2f}<br>"
            "t1*: %{customdata[3]:.0f} meses<br>"
            "t2*: %{customdata[4]:.0f} meses<extra></extra>"
        )
    ))

    # Marca o ponto ótimo global
    fig_curve.add_trace(go.Scatter(
        x=[a1_opt], y=[t_total_opt],
        mode="markers",
        name="Ótimo global",
        marker=dict(size=12, symbol="star"),
        hovertemplate=(
            "ÓTIMO GLOBAL<br>"
            f"a1={a1_opt:,.2f}<br>a2={a2_opt:,.2f}<br>"
            f"Ax1={Ax1_opt:,.2f}<br>Ax2={Ax2_opt:,.2f}<br>"
            f"t_total={t_total_opt:.0f} meses<extra></extra>"
        )
    ))

    fig_curve.update_layout(
        title="Curva: melhor tempo total vs a1 (com Ax1 ótimo por a1)",
        xaxis_title="a1 (aporte mensal no inv1)",
        yaxis_title="Tempo total mínimo (meses)"
    )

    fig_curve.show()


### 11 — Resumo final (com checagens)

In [12]:
# =========================
# Célula 11 — Resumo final (com checagens)
# =========================

# Calcula alguns números úteis
anos_total = t_total_sim / 12.0 if np.isfinite(t_total_sim) else np.inf  # converte meses -> anos
saldo_final_1 = df_traj["saldo1"].iloc[-1]                               # saldo final inv1
saldo_final_2 = df_traj["saldo2"].iloc[-1]                               # saldo final inv2
saldo_final_total = saldo_final_1 + saldo_final_2                        # saldo final total

# Data prevista para atingir as metas
data_final = data_usuario + timedelta(days=t_total_sim*30)
data_t1 = data_usuario + timedelta(days=t1_sim*30)
data_t2 = data_usuario + timedelta(days=t2_sim*30)

# Monta resumo em dataframe
resumo = pd.DataFrame({
    "Item": [
        "Taxa mensal i",
        "C1 (capital inicial inv1)", 
        "C2 (capital inicial inv2)",
        "AxT (extra total)", 
        "aT (aporte mensal total)",
        "Meta M1", 
        "Meta M2",
        "a1 ótimo", 
        "a2 ótimo",
        "Ax1 ótimo", 
        "Ax2 ótimo",
        "t1 (meses)",
        "Data prevista para atingir a meta 1",
        "t2 (meses)", 
        "Data prevista para atingir a meta 2",
        "t_total (meses)", 
        "t_total (anos)",
        "Data prevista para atingir as metas",
        "Saldo final inv1", 
        "Saldo final inv2", 
        "Saldo final total",
        
    ],
    "Valor": [
        i,
        C1, 
        C2,
        AxT, 
        aT,
        M1_meta, 
        M2_meta,
        a1_opt, 
        a2_opt,
        Ax1_opt, 
        Ax2_opt,
        t1_sim,
        data_t1.strftime("%Y-%m-%d"), 
        t2_sim,
        data_t2.strftime("%Y-%m-%d"), 
        t_total_sim, 
        anos_total,
        data_final.strftime("%Y-%m-%d"),
        saldo_final_1, 
        saldo_final_2, 
        saldo_final_total        
    ]
})

# Imprime resumo formatado
pd.set_option("display.float_format", lambda x: f"{x:,.6f}")  # formatação
resumo


,Item,Valor
0,Taxa mensal i,0.010979
1,C1 (capital inicial inv1),"166,358.790000"
2,C2 (capital inicial inv2),0.000000
3,AxT (extra total),"75,260.010000"
4,aT (aporte mensal total),"7,500.000000"
5,Meta M1,"450,000.000000"
6,Meta M2,"45,000.000000"
7,a1 ótimo,125.000000
8,a2 ótimo,"7,375.000000"
9,Ax1 ótimo,"67,734.009000"
